In [238]:
import sys
!{sys.executable} -m pip install geopy

In [239]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from geopy.geocoders import Nominatim
import time

In [240]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [276]:
df = pd.read_csv("../../output/preprocessed_1.csv", compression="gzip", parse_dates=["ts"])

In [277]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8381435 entries, 0 to 8381434
Data columns (total 12 columns):
 #   Column     Dtype         
---  ------     -----         
 0   ts         datetime64[ns]
 1   number     float64       
 2   pick_lat   float64       
 3   pick_lng   float64       
 4   drop_lat   float64       
 5   drop_lng   float64       
 6   hour       int64         
 7   minute     int64         
 8   day        int64         
 9   month      int64         
 10  year       int64         
 11  dayofweek  int64         
dtypes: datetime64[ns](1), float64(5), int64(6)
memory usage: 767.3 MB


In [278]:
df.head()

,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,hour,minute,day,month,year,dayofweek
0,2020-03-26 07:07:17,14626.0,12.313621,76.658195,12.287301,76.602280,7,7,26,3,2020,3
1,2020-03-26 07:32:27,85490.0,12.943947,77.560745,12.954014,77.543770,7,32,26,3,2020,3
2,2020-03-26 07:36:44,5408.0,12.899603,77.587300,12.934780,77.569950,7,36,26,3,2020,3
3,2020-03-26 07:38:00,58940.0,12.918229,77.607544,12.968971,77.636375,7,38,26,3,2020,3
4,2020-03-26 07:39:29,5408.0,12.899490,77.587270,12.934780,77.569950,7,39,26,3,2020,3


Now that we have done the basic cleanup of our raw data by (removing all the duplicate values) and (removing all the null values). Now, we are going to use the (good ride request) definition that the business management team gave us to follow.

### Data Cleaning with Business Understanding

Problem: There can be cases when a user requests a ride and their booking request is logged in our database. However, this user eventually re-books his/her ride due to various possible reasons such as:
- too long to wait, so want to try change the driver in an attempt to reduce waiting time 
- driver refused the booking for some reason
- user refused the booking for some reason (ex: doesn't like the driver because he/she has a low rating)
- user by mistake added wrong (pick up or drop off) locations

Using the (good ride request) definitions we will be handling each case by case - to obtain a dataset only with the good/real bookings placed by users to go from one location to another, so we want the real demand, not fraud bookings, not duplicate bookings, not repeated booking - so we need to remove all those entries. Because we want to know the exact demand and create a model on top of that data.

In [279]:
df.sort_values(by=["number", "ts"], inplace=True, ascending=True)
df.reset_index(inplace=True)

In [280]:
df["booking_timestamp"] = df["ts"].values.astype(np.int64) // 10 ** 9

In [281]:
df.head(50)

,index,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,hour,minute,day,month,year,dayofweek,booking_timestamp
0,2393751,2020-10-10 07:34:16,-1.0,12.975773,77.571070,12.878468,77.445330,7,34,10,10,2020,5,1602315256
1,2425471,2020-10-11 08:23:42,-1.0,12.930813,77.609530,12.960320,77.587210,8,23,11,10,2020,6,1602404622
2,2425472,2020-10-11 08:23:50,-1.0,12.930813,77.609530,12.960320,77.587210,8,23,11,10,2020,6,1602404630
3,2425473,2020-10-11 08:23:51,-1.0,12.930813,77.609530,12.960320,77.587210,8,23,11,10,2020,6,1602404631
4,2425474,2020-10-11 08:23:54,-1.0,12.930813,77.609530,12.960320,77.587210,8,23,11,10,2020,6,1602404634
5,2425475,2020-10-11 08:23:56,-1.0,12.930813,77.609530,12.960320,77.587210,8,23,11,10,2020,6,1602404636
6,2425654,2020-10-11 11:57:17,-1.0,12.960213,77.587460,12.930824,77.609610,11,57,11,10,2020,6,1602417437
7,2425655,2020-10-11 11:57:31,-1.0,12.960213,77.587460,12.930824,77.609610,11,57,11,10,2020,6,1602417451
8,2520953,2020-10-16 17:51:07,-1.0,12.924353,77.549410,12.932216,77.581825,17,51,16,10,2020,4,1602870667
9,2520954,2020-10-16 17:51:25,-1.0,12.924353,77.549410,12.932216,77.581825,17,51,16,10,2020,4,1602870685


To interpret this data, let's take the firs row:
user with id "2393751" made a ride request from pick up location being (lat=12.975773, lng=77.571070) to drop off location being (lat=12.878468, lng=77.445330) at the timestamp (1602315256)

**Case 1: Handle rebooking again to the same location** => Keep only one the last request of the same user to the same pickup latitude/longitude in 1 hour time frame since the first ride request

(ex: user could be booking rides again and again from the same pickup location within 1 hour from his first booking, we will remove all those cases except for the last one)

Now, we will perform a shift of the entire booking timestamp with groupby on the user id and then we will calculate the time differce between two bookings, in other words we are taking a lag of one step:

In [282]:
df["shift_booking_ts"] = df.groupby("number")["booking_timestamp"].shift(1)
df["shift_booking_ts"].fillna(0, inplace=True) # for each user, there will be one missing value created in shift_booking_ts column, so we fill it with 0
df["shift_booking_ts"] = df["shift_booking_ts"].astype('int64')
df.head(900)

,index,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,hour,minute,day,month,year,dayofweek,booking_timestamp,shift_booking_ts
0,2393751,2020-10-10 07:34:16,-1.0,12.975773,77.571070,12.878468,77.445330,7,34,10,10,2020,5,1602315256,0
1,2425471,2020-10-11 08:23:42,-1.0,12.930813,77.609530,12.960320,77.587210,8,23,11,10,2020,6,1602404622,1602315256
2,2425472,2020-10-11 08:23:50,-1.0,12.930813,77.609530,12.960320,77.587210,8,23,11,10,2020,6,1602404630,1602404622
3,2425473,2020-10-11 08:23:51,-1.0,12.930813,77.609530,12.960320,77.587210,8,23,11,10,2020,6,1602404631,1602404630
4,2425474,2020-10-11 08:23:54,-1.0,12.930813,77.609530,12.960320,77.587210,8,23,11,10,2020,6,1602404634,1602404631
5,2425475,2020-10-11 08:23:56,-1.0,12.930813,77.609530,12.960320,77.587210,8,23,11,10,2020,6,1602404636,1602404634
6,2425654,2020-10-11 11:57:17,-1.0,12.960213,77.587460,12.930824,77.609610,11,57,11,10,2020,6,1602417437,1602404636
7,2425655,2020-10-11 11:57:31,-1.0,12.960213,77.587460,12.930824,77.609610,11,57,11,10,2020,6,1602417451,1602417437
8,2520953,2020-10-16 17:51:07,-1.0,12.924353,77.549410,12.932216,77.581825,17,51,16,10,2020,4,1602870667,1602417451
9,2520954,2020-10-16 17:51:25,-1.0,12.924353,77.549410,12.932216,77.581825,17,51,16,10,2020,4,1602870685,1602870667


In [283]:
# Calculating the difference between two consecutive booking
# df["booking_timestamp"]-df["shift_booking_ts"] => difference in time between current request and previous request that a specific user made
# note: by having a zero filled for empty value for 'shift_booking_ts', we will never compare time differences in booking made by different users
# so comparisons in request times are only made within each individual user

# Difference in hours
df["booking_time_diff_hr"] = round((df["booking_timestamp"]-df["shift_booking_ts"])//3600)
# Difference in minutes
df["booking_time_diff_min"] = round((df["booking_timestamp"]-df["shift_booking_ts"])//60)

In [284]:
# Booking time differences in minutes
df["booking_time_diff_min"].value_counts().to_dict()

{0: 2867802,
 1: 799993,
 2: 307831,
 3: 167954,
 4: 109141,
 5: 77918,
 6: 60416,
 7: 48342,
 8: 40404,
 9: 34407,
 10: 29977,
 11: 26091,
 12: 23336,
 13: 20839,
 14: 18945,
 15: 16872,
 16: 15467,
 17: 13885,
 18: 12804,
 19: 11870,
 20: 10869,
 21: 10105,
 22: 9397,
 23: 8810,
 24: 8185,
 25: 7752,
 26: 7255,
 27: 6898,
 28: 6610,
 29: 6263,
 30: 6034,
 31: 5773,
 1439: 5725,
 1440: 5712,
 32: 5702,
 1438: 5489,
 1437: 5402,
 1441: 5391,
 33: 5349,
 1442: 5303,
 1443: 5178,
 34: 5094,
 1436: 5076,
 35: 4965,
 1435: 4962,
 36: 4907,
 1444: 4861,
 38: 4780,
 37: 4757,
 1434: 4724,
 1433: 4662,
 1445: 4581,
 39: 4542,
 40: 4464,
 41: 4416,
 1432: 4407,
 1446: 4401,
 42: 4292,
 43: 4213,
 1431: 4170,
 44: 4133,
 1447: 4123,
 1430: 4038,
 45: 4016,
 1448: 3993,
 47: 3989,
 46: 3977,
 48: 3931,
 50: 3872,
 1449: 3838,
 1429: 3836,
 51: 3817,
 49: 3807,
 53: 3693,
 52: 3659,
 1428: 3644,
 54: 3643,
 55: 3624,
 1427: 3611,
 56: 3577,
 59: 3563,
 1450: 3552,
 1426: 3521,
 63: 3449,
 57: 344

In [285]:
# Booking time differences in hours
df["booking_time_diff_hr"].value_counts().head(10)

0     4915909
1      164914
23     152962
24     132454
2      118199
9      104854
3       90789
10      86794
8       84454
14      83715
Name: booking_time_diff_hr, dtype: int64

In [286]:
df.shape[0]

8381435

In [287]:
df.duplicated(subset=["number", "pick_lat", "pick_lng"], keep=False).value_counts()

True     5177260
False    3204175
dtype: int64

In [288]:
(df["booking_time_diff_hr"] <= 1).value_counts()

True     5080823
False    3300612
Name: booking_time_diff_hr, dtype: int64

In [289]:
res = (df.duplicated(subset=["number", "pick_lat", "pick_lng"], keep=False) & (df["booking_time_diff_hr"] <= 1))
res.value_counts()

False    4331269
True     4050166
dtype: int64

In [290]:
rows_to_remove_df = df[res]
rows_to_remove_df.head()

# this dataset contains all rows where the same user repeatedly makes several bookings from the same location within less than an hour

,index,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,hour,minute,day,month,year,dayofweek,booking_timestamp,shift_booking_ts,booking_time_diff_hr,booking_time_diff_min
2,2425472,2020-10-11 08:23:50,-1.0,12.930813,77.60953,12.960320,77.58721,8,23,11,10,2020,6,1602404630,1602404622,0,0
3,2425473,2020-10-11 08:23:51,-1.0,12.930813,77.60953,12.960320,77.58721,8,23,11,10,2020,6,1602404631,1602404630,0,0
4,2425474,2020-10-11 08:23:54,-1.0,12.930813,77.60953,12.960320,77.58721,8,23,11,10,2020,6,1602404634,1602404631,0,0
5,2425475,2020-10-11 08:23:56,-1.0,12.930813,77.60953,12.960320,77.58721,8,23,11,10,2020,6,1602404636,1602404634,0,0
7,2425655,2020-10-11 11:57:31,-1.0,12.960213,77.58746,12.930824,77.60961,11,57,11,10,2020,6,1602417451,1602417437,0,0


In [291]:
display(df.shape[0])
df.drop(rows_to_remove_df.index, axis=0, inplace=True)
display(df.shape[0])

8381435

4331269

Case 2: Handle location entry mistake => Keep only last request of user within 5-8 minutes of first booking request

- Person booking a ride would generally book a ride that would take 5-8 minutes of ride time.
- Calculate distance between (pick up and drop off) locations. Based on the distance and request time, we will remove bad entries.

Case 2.1: Remove entries with (Pick up and Drop off) latitude/longitude distance of less than 50 meters = 0.05km as no user would like to ride for just a 50 meters trip

Managment has observed that when a user books a ride, generally a rider takes 3-4 minutes to arrive to the pickup location from this current location. So 4 minutes is the pick up time generally taken by a rider. Another 4 minutes would be the ride time. So it would take approx 8 minutes in total for a user to move from pick up location to drop off location since the moment of making a booking. Within this 8 minutes a user cannot be booking new rides again and again. So, the management team have considered that if there's a ride booking happening within the first 8 minutes, we should remove any additional requests being made while this ride is taking place.

In [292]:
booking_less_than_8_min_diff_bool = df["booking_time_diff_min"] < 8
booking_less_than_8_min_diff_bool.value_counts()

print("Number of rides booked by same customer within 8 minutes times: {}".format((booking_less_than_8_min_diff_bool==True).sum()))

rows_to_remove = df[booking_less_than_8_min_diff_bool]

Number of rides booked by same customer within 8 minutes times: 609163


In [293]:
df.drop(rows_to_remove.index, axis=0, inplace=True)

In [294]:
display(df.shape[0])

3722106

Assuming that earth is an ellipsoid, we calculating geodesic distance between (pick up and drop off) (latitude and longitude) in the following way:

In [295]:
def geodesic_distance(pick_lat, pick_lng, drop_lat, drop_lng):
    # 1 mile = 1.60934
    return round(geodesic((pick_lat, pick_lng), (drop_lat, drop_lng)).miles * 1.60934, 2)

# np.vectorize is a vectorized way of calling this function, which is more efficient in terms of performance
df["geodesic_distance"] = np.vectorize(geodesic_distance)(df["pick_lat"], df["pick_lng"], df["drop_lat"], df["drop_lng"])

The method above can take between 5-30 minutes to calculate depending on your computer

When a person books a ride, generally we've seen that the ride has a distance of about 0.05 kilometers = 50 meters. Therfore, less than 50m travel distance doesn't make sence for anyone to book a ride. We want to remove entries who have been making booking for such short distances of 50m or less, because 50m is a very short distance and it doesn't make sense for them to book a ride when they can simply walk. So all these entries will be considered as fraud rides due to some problem in the system which led to these entries being logged or coordinates (lat, lng) of (pick and drop( weren't captured correctly.

In [296]:
short_distance_bool = df["geodesic_distance"] <= 0.05
(short_distance_bool).value_counts()

False    3707659
True       14447
Name: geodesic_distance, dtype: int64

In [297]:
df[short_distance_bool]["geodesic_distance"].value_counts()

0.00    6615
0.01    2490
0.02    2010
0.03    1369
0.04    1087
0.05     876
Name: geodesic_distance, dtype: int64

In [298]:
print("Number of rides requests less than 50 meters: {} number of rides from a total of: {} number of rides".format(
    short_distance_bool.sum(),
    short_distance_bool.count()))

Number of rides requests less than 50 meters: 14447 number of rides from a total of: 3722106 number of rides


In [301]:
df.drop(df[short_distance_bool].index, axis=0, inplace=True)

/var/folders/5p/g86ptb251wz01xb5n3ngh_gh0000gn/T/ipykernel_43473/1628738162.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.drop(df[short_distance_bool].index, axis=0, inplace=True)


In [302]:
df.shape[0]

3707659

In [304]:
df.to_csv("../../output/preprocessing_2.csv", index=False, compression="gzip")

Case 3: Handle booking location outside operation zone of the company

In [414]:
ndf = pd.read_csv("../../output/preprocessing_2.csv", compression="gzip", parse_dates=["ts"], index_col="index")

So far we have removed based on:
- (geodistic distance)
- (when multiple bookings are made by the same user within 8 minutes after the previous booking)
- (when multiple booking are made by the same user with the same pick up location within 1 hour after the previous booking)

Consequently, this is the dataset we currently have:

In [415]:
ndf.head()

,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,hour,minute,day,month,year,dayofweek,booking_timestamp,shift_booking_ts,booking_time_diff_hr,booking_time_diff_min,geodesic_distance
index,,,,,,,,,,,,,,,,,
2393751,2020-10-10 07:34:16,-1.0,12.975773,77.57107,12.878468,77.445330,7,34,10,10,2020,5,1602315256,0,445087,26705254,17.38
2425471,2020-10-11 08:23:42,-1.0,12.930813,77.60953,12.960320,77.587210,8,23,11,10,2020,6,1602404622,1602315256,24,1489,4.06
2425654,2020-10-11 11:57:17,-1.0,12.960213,77.58746,12.930824,77.609610,11,57,11,10,2020,6,1602417437,1602404636,3,213,4.04
2520953,2020-10-16 17:51:07,-1.0,12.924353,77.54941,12.932216,77.581825,17,51,16,10,2020,4,1602870667,1602417451,125,7553,3.62
2716681,2020-10-30 09:00:44,-1.0,12.945731,77.62250,12.973030,77.616840,9,0,30,10,2020,4,1604048444,1602870685,327,19629,3.08


Management have also shared with us that Bangalore is the city where most rides occur and that sometimes rides come from non-operational regions. So, all rides coming from operational regions have to be removed.

- India: 'boundingbox': ['6.2325274', '35.6745457', '68.1113787', '97.395561']
- Bangalore:'boundingbox': ['12.8340125', '13.1436649', '77.4601025', '77.7840515']
- Karnataka: 'boundingbox': ['11.5945587', '18.4767308', '74.0543908', '78.588083']

Source: openstreemap

In [416]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="aoo")
location_India = geolocator.geocode("India")
location_Bangalore = geolocator.geocode("Bangalore")
location_Karnataka = geolocator.geocode("Karnataka")

In [417]:
boundaries_India = np.array(location_India.raw["boundingbox"]).astype(float)
boundaries_Bangalore = np.array(location_Bangalore.raw["boundingbox"]).astype(float)
boundaries_Karnataka = np.array(location_Karnataka.raw["boundingbox"]).astype(float)

In [418]:
boundaries_India

array([ 6.5531169, 35.6745457, 67.9544415, 97.395561 ])

Now, we want to remove the entries that have (pick up) or (drop off) locations outside of the regions where our company operates:

In [419]:
# How many (pick ups and drop offs) were outside India?
outside_India = ndf[
    (ndf["pick_lat"] <= boundaries_India[0]) |
    (ndf["pick_lat"] >= boundaries_India[1]) |
    (ndf["pick_lng"] <= boundaries_India[2]) |
    (ndf["pick_lng"] >= boundaries_India[3]) |
    (ndf["drop_lat"] <= boundaries_India[0]) | 
    (ndf["drop_lat"] >= boundaries_India[1]) | 
    (ndf["drop_lng"] <= boundaries_India[2]) | 
    (ndf["drop_lng"] >= boundaries_India[3])
]
print(outside_India.shape)
display(outside_India)

(641, 17)


,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,hour,minute,day,month,year,dayofweek,booking_timestamp,shift_booking_ts,booking_time_diff_hr,booking_time_diff_min,geodesic_distance
index,,,,,,,,,,,,,,,,,
747130,2020-06-22 07:19:06,80.0,12.926255,77.616400,0.000000e+00,0.000000e+00,7,19,22,6,2020,0,1592810346,1590934080,521,31271,8674.57
2522789,2020-10-16 21:52:28,297.0,12.958342,77.517876,3.890229e+00,-3.344596e+01,21,52,16,10,2020,4,1602885148,1602884658,0,8,12181.79
4384931,2021-01-07 13:17:11,345.0,12.824208,77.684840,5.347187e+01,-1.021332e+02,13,17,7,1,2021,3,1610025431,1609843045,50,3039,12658.48
8363378,2021-03-26 20:16:01,485.0,12.932975,77.536230,2.819774e+01,1.129942e+02,20,16,26,3,2021,4,1616789761,1616697903,25,1530,4043.95
1800835,2020-09-09 09:36:43,641.0,12.896367,77.623800,0.000000e+00,0.000000e+00,9,36,9,9,2020,2,1599644203,0,444345,26660736,8675.21
1775452,2020-09-08 11:13:17,1440.0,12.963732,77.584320,0.000000e+00,0.000000e+00,11,13,8,9,2020,1,1599563597,1591096285,2352,141121,8671.29
1550722,2020-08-30 23:22:17,1711.0,12.933468,77.601370,1.400000e-45,1.400000e-45,23,22,30,8,2020,6,1598829737,1597778313,292,17523,8672.97
1531967,2020-08-28 21:02:42,2012.0,12.943290,77.558500,1.400000e-45,1.400000e-45,21,2,28,8,2020,4,1598648562,0,444069,26644142,8668.38
2582928,2020-10-19 17:24:45,2196.0,12.973126,77.583150,1.400000e-45,1.400000e-45,17,24,19,10,2020,0,1603128285,1603006478,33,2030,8671.21


We see that there're are 641 cases request orders that have occured outside India.

If we observe the latitude and lontitude of these entries, some of them are equal to zero, so seems like some crash/error/bug in the system has occured when these entries were logged. So, we have to remove these entries. as otherwise they would negatively affect the performance of our machine learning model.

In [420]:
# Dropping the booking that have (pick up or drop off) locations outside of India

ndf.drop(outside_India.index, axis=0, inplace=True)

In [421]:
print(ndf.shape)
display(ndf.head())

(3707018, 17)


,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,hour,minute,day,month,year,dayofweek,booking_timestamp,shift_booking_ts,booking_time_diff_hr,booking_time_diff_min,geodesic_distance
index,,,,,,,,,,,,,,,,,
2393751,2020-10-10 07:34:16,-1.0,12.975773,77.57107,12.878468,77.445330,7,34,10,10,2020,5,1602315256,0,445087,26705254,17.38
2425471,2020-10-11 08:23:42,-1.0,12.930813,77.60953,12.960320,77.587210,8,23,11,10,2020,6,1602404622,1602315256,24,1489,4.06
2425654,2020-10-11 11:57:17,-1.0,12.960213,77.58746,12.930824,77.609610,11,57,11,10,2020,6,1602417437,1602404636,3,213,4.04
2520953,2020-10-16 17:51:07,-1.0,12.924353,77.54941,12.932216,77.581825,17,51,16,10,2020,4,1602870667,1602417451,125,7553,3.62
2716681,2020-10-30 09:00:44,-1.0,12.945731,77.62250,12.973030,77.616840,9,0,30,10,2020,4,1604048444,1602870685,327,19629,3.08


In [422]:
boundaries_Bangalore

array([12.8167936, 13.1367936, 77.430082 , 77.750082 ])

In [423]:
# How many (pick ups and drop offs) were outside Banglore?
pck_outside_bng = ndf[(ndf.pick_lat<=boundaries_Bangalore[0]) | (ndf.pick_lat>=boundaries_Bangalore[1]) | (ndf.pick_lng<=boundaries_Bangalore[2]) | (ndf.pick_lng>=boundaries_Bangalore[3])]
drp_outside_bng = ndf[(ndf.drop_lat<=boundaries_Bangalore[0]) | (ndf.drop_lat>=boundaries_Bangalore[1]) | (ndf.drop_lng<=boundaries_Bangalore[2]) | (ndf.drop_lng>=boundaries_Bangalore[3])]
print("Number of Pickup Requests Outside Bangalore: ",len(pck_outside_bng))
print("Number of Customers pickup outside Bangalore: ",len(np.unique(pck_outside_bng['number'].values)))

print("Number of Drops Requests Outside Bangalore: ",len(drp_outside_bng))
print("Number of Customers Drop outside Bangalore: ",len(np.unique(drp_outside_bng['number'].values)))

Number of Pickup Requests Outside Bangalore:  159783
Number of Customers pickup outside Bangalore:  21422
Number of Drops Requests Outside Bangalore:  173221
Number of Customers Drop outside Bangalore:  28123


In [424]:
boundaries_Karnataka

array([11.5945587, 18.4766494, 74.0543908, 78.588083 ])

In [425]:
# How many (pick ups and drop offs) were outside Karnataka?
pck_outside_KA = ndf[(ndf.pick_lat<=boundaries_Karnataka[0]) | (ndf.pick_lat>=boundaries_Karnataka[1]) | (ndf.pick_lng<=boundaries_Karnataka[2]) | (ndf.pick_lng>=boundaries_Karnataka[3])]
drp_outside_KA = ndf[(ndf.drop_lat<=boundaries_Karnataka[0]) | (ndf.drop_lat>=boundaries_Karnataka[1]) | (ndf.drop_lng<=boundaries_Karnataka[2]) | (ndf.drop_lng>=boundaries_Karnataka[3])]
print("Pickups Outisde KA: {} \nDrop outside KA: {}".format(len(pck_outside_KA),len(drp_outside_KA)))
print("Number of Customers Drop outside KA: ",len(np.unique(drp_outside_KA['number'].values)))
print("Number of Customers pickup outside KA: ",len(np.unique(pck_outside_KA['number'].values)))

Pickups Outisde KA: 38785 
Drop outside KA: 39562
Number of Customers Drop outside KA:  6914
Number of Customers pickup outside KA:  6300


The data above is telling us that most of the booking outside of operating regions occur outside of the Banglore region. 

In [426]:
total_ride_outside_KA = ndf[(ndf.pick_lat<=boundaries_Karnataka[0]) | (ndf.pick_lat>=boundaries_Karnataka[1]) | (ndf.pick_lng<=boundaries_Karnataka[2]) | (ndf.pick_lng>=boundaries_Karnataka[3]) | (ndf.drop_lat<=boundaries_Karnataka[0]) | (ndf.drop_lat>=boundaries_Karnataka[1]) | (ndf.drop_lng<=boundaries_Karnataka[2]) | (ndf.drop_lng>=boundaries_Karnataka[3])]

In [427]:
print("Total Ride Outside Karnataka: {}".format(len(total_ride_outside_KA)))

Total Ride Outside Karnataka: 39609


In [428]:
# Since OLA doesn't support intercity ridesm, all the rides happen within the particular city.
# Therefore, for all the rides happening outside Karnatka with geodestic distance greater than 500km seems 
# suspicious as it is unlikely that someone will book a ride on a bike for more than 500 km.
# So, we will remove them from the dataset

suspected_bad_rides = total_ride_outside_KA[total_ride_outside_KA["geodesic_distance"] > 500]
print(len(suspected_bad_rides))
suspected_bad_rides.head()

506


,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,hour,minute,day,month,year,dayofweek,booking_timestamp,shift_booking_ts,booking_time_diff_hr,booking_time_diff_min,geodesic_distance
index,,,,,,,,,,,,,,,,,
4807812,2021-01-21 23:18:35,0.0,12.958837,77.644485,21.149794,82.782000,23,18,21,1,2021,3,1611271115,1611219970,14,852,1058.39
1770243,2020-09-08 14:49:08,154.0,12.922410,77.568270,25.549080,84.655800,14,49,8,9,2020,1,1599576548,1599496883,22,1327,1583.00
590302,2020-06-04 18:13:54,878.0,12.970724,77.582420,21.504763,80.115160,18,13,4,6,2020,3,1591294434,1591266878,7,459,982.06
123119,2020-04-07 11:09:24,913.0,13.004375,77.555210,19.075983,72.877655,11,9,7,4,2020,1,1586257764,1586102620,43,2585,837.66
290905,2020-04-29 23:11:41,913.0,13.013597,77.545740,22.837053,69.724560,23,11,29,4,2020,2,1588201901,1588159989,11,698,1366.20


In [429]:
display(ndf.head())

,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,hour,minute,day,month,year,dayofweek,booking_timestamp,shift_booking_ts,booking_time_diff_hr,booking_time_diff_min,geodesic_distance
index,,,,,,,,,,,,,,,,,
2393751,2020-10-10 07:34:16,-1.0,12.975773,77.57107,12.878468,77.445330,7,34,10,10,2020,5,1602315256,0,445087,26705254,17.38
2425471,2020-10-11 08:23:42,-1.0,12.930813,77.60953,12.960320,77.587210,8,23,11,10,2020,6,1602404622,1602315256,24,1489,4.06
2425654,2020-10-11 11:57:17,-1.0,12.960213,77.58746,12.930824,77.609610,11,57,11,10,2020,6,1602417437,1602404636,3,213,4.04
2520953,2020-10-16 17:51:07,-1.0,12.924353,77.54941,12.932216,77.581825,17,51,16,10,2020,4,1602870667,1602417451,125,7553,3.62
2716681,2020-10-30 09:00:44,-1.0,12.945731,77.62250,12.973030,77.616840,9,0,30,10,2020,4,1604048444,1602870685,327,19629,3.08


In [430]:
ndf.drop(suspected_bad_rides.index, axis=0, inplace=True)

In [431]:
total_ride_outside_BNG = ndf[(ndf.pick_lat<=boundaries_Bangalore[0]) | (ndf.pick_lat>=boundaries_Bangalore[1]) | (ndf.pick_lng<=boundaries_Bangalore[2]) | (ndf.pick_lng>=boundaries_Bangalore[3]) | (ndf.drop_lat<=boundaries_Bangalore[0]) | (ndf.drop_lat>=boundaries_Bangalore[1]) | (ndf.drop_lng<=boundaries_Bangalore[2]) | (ndf.drop_lng>=boundaries_Bangalore[3])]

In [432]:
print("Total Ride Outside Bangalore: {}".format(len(total_ride_outside_BNG)))

Total Ride Outside Bangalore: 179034


In [433]:
suspected_bad_rides = total_ride_outside_BNG[total_ride_outside_BNG["geodesic_distance"] > 500]
print(len(suspected_bad_rides))
suspected_bad_rides.head()

102


,ts,number,pick_lat,pick_lng,drop_lat,drop_lng,hour,minute,day,month,year,dayofweek,booking_timestamp,shift_booking_ts,booking_time_diff_hr,booking_time_diff_min,geodesic_distance
index,,,,,,,,,,,,,,,,,
266702,2020-04-26 22:29:46,3728.0,12.919394,77.614250,17.411560,78.423690,22,29,26,4,2020,6,1587940186,1587939626,0,9,504.61
3294585,2020-11-23 20:45:34,4017.0,12.921143,77.570320,17.357449,78.486946,20,45,23,11,2020,0,1606164334,1604758367,390,23432,500.66
2377785,2020-10-09 11:26:25,4050.0,12.900018,77.634895,17.428600,78.358870,11,26,9,10,2020,4,1602242785,1602210339,9,540,507.09
6816484,2021-03-02 11:48:00,5470.0,12.964750,77.584320,17.443650,78.445820,11,48,2,3,2021,1,1614685680,1614676301,2,156,504.16
7224329,2021-03-09 11:58:34,8583.0,12.957649,77.621200,17.433240,78.503300,11,58,9,3,2021,1,1615291114,1615282517,2,143,504.21


In [434]:
ndf.drop(suspected_bad_rides.index, axis=0, inplace=True)

In [435]:
print("Number of Good Ride Requests: {}".format(len(ndf)))

Number of Good Ride Requests: 3706410


In [439]:
dataset = df[['ts', 'number', 'pick_lat','pick_lng','drop_lat','drop_lng','geodesic_distance','hour','minute','day','month','year','dayofweek','booking_timestamp','booking_time_diff_hr', 'booking_time_diff_min']]
dataset.to_csv('../../output/clean_data.csv',index = False, compression = 'gzip')